In [1]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format

# 1. Load the data

In [3]:
data=pd.read_csv('C:/Users/tsb7592/Downloads/Basic_Desc_output.csv')

In [4]:
data['PIN']=data['PIN'].str.upper()

In [5]:
data.head()

,PIN,DeviceID,Assessment Name,Inst,ItemID,Response,Score,Position,ResponseTime,DateCreated,InstStarted,InstEnded,Age_month,Age_group
0,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Title,1,NaN,1,NaN,2021-12-05 08:30:42,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
1,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Instructions,1,NaN,2,NaN,2021-12-05 08:30:43,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
2,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL13,2,0.00,3,NaN,2021-12-05 08:32:53,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
3,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL10,1,1.00,4,NaN,2021-12-05 08:32:59,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
4,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL15,2,0.00,5,NaN,2021-12-05 08:33:18,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"


# 2. Filter the inst to be IBQ

In [6]:
df= data[data.Inst=='Infant Behavior Questionnaire - Revised - Very Short Form']

In [7]:
df.PIN=df.PIN.str[:5]

In [8]:
df.Response =pd.to_numeric(df.Response)
df.Score =pd.to_numeric(df.Score)

In [9]:
df.describe()

,Response,Score,Position,ResponseTime,Age_month
count,1482.00,1357.00,1482.00,1406.00,897.00
mean,4.64,4.76,20.00,13.97,373.48
std,2.10,1.89,11.26,51.86,67.90
min,1.00,1.00,1.00,0.51,257.00
25%,3.00,3.00,10.00,4.47,320.00
50%,5.00,5.00,20.00,6.43,371.00
75%,6.00,6.00,30.00,10.91,445.00
max,8.00,7.00,39.00,1071.29,493.00


# 3. Rescore all items to ensure appropriate response coding

In [10]:
import numpy as np
#The score should equal the response for all items EXCEPT SelfReg_ibqrvsh11, for which the score should be 8-response 
df['Score'] = np.where(df.ItemID=='SelfReg_ibqrvsh11',8-df.Response, df.Response)

# 4. Calculate the three composite scores

In [11]:
a= ['SelfReg_ibqrvsh1','SelfReg_ibqrvsh2',
    'SelfReg_ibqrvsh7','SelfReg_ibqrvsh8', 
    'SelfReg_ibqrvsh13','SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15',
    'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21',
    'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27' ,
    'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

b= ['SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
    'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
    'SelfReg_ibqrvsh16', 'SelfReg_ibqrvsh17',  
    'SelfReg_ibqrvsh22', 'SelfReg_ibqrvsh23',  
    'SelfReg_ibqrvsh28', 'SelfReg_ibqrvsh29', 
    'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33']


c= [ 'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6',
    'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 
    'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
    'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
    'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
      'SelfReg_ibqrvsh34', 'SelfReg_ibqrvsh35']

In [12]:
SelfReg_SurgIBQR= df[df.ItemID.isin(a)].groupby('PIN')['Score'].mean()
SelfReg_NegAffIBQR= df[df.ItemID.isin(b)].groupby('PIN')['Score'].mean()
SelfReg_EffContIBQR= df[df.ItemID.isin(c)].groupby('PIN')['Score'].mean()


In [13]:
comp =[SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR]

In [14]:
cdf = pd.concat(comp,axis=1)
cdf.columns = ['SelfReg_SurgIBQR','SelfReg_NegAffIBQR','SelfReg_EffContIBQR']

In [15]:
cdf.corr()

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
SelfReg_SurgIBQR,1.00,0.40,0.41
SelfReg_NegAffIBQR,0.40,1.00,0.04
SelfReg_EffContIBQR,0.41,0.04,1.00


# 5. Calculate item-rest correlations for every item – e.g., what is the Pearson correlation for SelfReg_ibqrvsh1 with the average score for the 12 other items on SelfReg_SurgIQBR. There should be 37 item-rest correlations. 

In [16]:
items=['SelfReg_ibqrvsh1', 'SelfReg_ibqrvsh2', 'SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
       'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6', 'SelfReg_ibqrvsh7',
       'SelfReg_ibqrvsh8', 'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
       'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 'SelfReg_ibqrvsh13',
       'SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15', 'SelfReg_ibqrvsh16',
       'SelfReg_ibqrvsh17', 'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
       'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21', 'SelfReg_ibqrvsh22',
       'SelfReg_ibqrvsh23', 'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
       'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27', 'SelfReg_ibqrvsh28',
       'SelfReg_ibqrvsh29', 'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
       'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33', 'SelfReg_ibqrvsh34',
       'SelfReg_ibqrvsh35', 'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

In [17]:
from scipy.stats import pearsonr

def item_rest(itemlist):

    for i in range(len(itemlist)):
        rest = itemlist.copy()
        rest.pop(i)
        list1 = df[df.ItemID==itemlist[i]].groupby('PIN')['Score'].mean()
        list2  = df[df.ItemID.isin(rest)].groupby('PIN')['Score'].mean()
    
    # Apply the pearsonr()
        corr, _ = pearsonr(list1, list2)
        print(itemlist[i], ': Pearsons correlation is %.3f ' % corr)

In [18]:
item_rest(a)

SelfReg_ibqrvsh1 : Pearsons correlation is 0.237 
SelfReg_ibqrvsh2 : Pearsons correlation is 0.180 
SelfReg_ibqrvsh7 : Pearsons correlation is 0.446 
SelfReg_ibqrvsh8 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh13 : Pearsons correlation is 0.196 
SelfReg_ibqrvsh14 : Pearsons correlation is 0.285 
SelfReg_ibqrvsh15 : Pearsons correlation is 0.155 
SelfReg_ibqrvsh20 : Pearsons correlation is 0.227 
SelfReg_ibqrvsh21 : Pearsons correlation is 0.242 
SelfReg_ibqrvsh26 : Pearsons correlation is 0.376 
SelfReg_ibqrvsh27 : Pearsons correlation is 0.360 
SelfReg_ibqrvsh36 : Pearsons correlation is 0.348 
SelfReg_ibqrvsh37 : Pearsons correlation is 0.197 


In [19]:
item_rest(b)

SelfReg_ibqrvsh3 : Pearsons correlation is 0.474 
SelfReg_ibqrvsh4 : Pearsons correlation is 0.603 
SelfReg_ibqrvsh9 : Pearsons correlation is 0.343 
SelfReg_ibqrvsh10 : Pearsons correlation is 0.265 
SelfReg_ibqrvsh16 : Pearsons correlation is 0.467 
SelfReg_ibqrvsh17 : Pearsons correlation is 0.326 
SelfReg_ibqrvsh22 : Pearsons correlation is -0.052 
SelfReg_ibqrvsh23 : Pearsons correlation is 0.548 
SelfReg_ibqrvsh28 : Pearsons correlation is 0.373 
SelfReg_ibqrvsh29 : Pearsons correlation is 0.409 
SelfReg_ibqrvsh32 : Pearsons correlation is 0.406 
SelfReg_ibqrvsh33 : Pearsons correlation is 0.448 


In [20]:
item_rest(c)

SelfReg_ibqrvsh5 : Pearsons correlation is 0.262 
SelfReg_ibqrvsh6 : Pearsons correlation is 0.312 
SelfReg_ibqrvsh11 : Pearsons correlation is 0.179 
SelfReg_ibqrvsh12 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh18 : Pearsons correlation is 0.530 
SelfReg_ibqrvsh19 : Pearsons correlation is 0.445 
SelfReg_ibqrvsh24 : Pearsons correlation is 0.411 
SelfReg_ibqrvsh25 : Pearsons correlation is 0.532 
SelfReg_ibqrvsh30 : Pearsons correlation is 0.437 
SelfReg_ibqrvsh31 : Pearsons correlation is 0.466 
SelfReg_ibqrvsh34 : Pearsons correlation is 0.510 
SelfReg_ibqrvsh35 : Pearsons correlation is 0.456 


# 6. Describe each of the 3 composite scores overall and by age group. NOTE: the Infant Behavior Questionnaire is not applicable to all age groups. 

In [21]:
cdf=cdf.reset_index()

In [22]:
spin = cdf.PIN.to_list()
cpin = []
for pin in spin:
    pin+='C'
    cpin.append(pin)

In [23]:
len(spin)

37

In [24]:
child =data[data.PIN.isin(cpin)]

In [25]:
child.PIN=child.PIN.str[:5]

In [26]:
child[child.Age_group.notnull()]['PIN'].nunique()

33

In [27]:
de = pd.merge(cdf, child[['PIN','Age_month','Age_group']], on=['PIN'], how ='left')

In [28]:
de

,PIN,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR,Age_month,Age_group
0,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
1,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
2,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
3,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
4,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
...,...,...,...,...,...,...
8843,WI032,4.62,4.75,4.67,8.00,"(6, 8]"
8844,WI032,4.62,4.75,4.67,8.00,"(6, 8]"
8845,WI032,4.62,4.75,4.67,8.00,"(6, 8]"
8846,WI032,4.62,4.75,4.67,8.00,"(6, 8]"


In [29]:
de.head()

,PIN,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR,Age_month,Age_group
0,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
1,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
2,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
3,MN001,4.62,4.50,6.42,4.00,"(3, 4]"
4,MN001,4.62,4.50,6.42,4.00,"(3, 4]"


In [30]:
de.groupby(['Age_group'])['SelfReg_SurgIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(10, 12]",1600.00,5.79,0.92,4.15,5.38,5.69,6.62,6.92
"(12, 15]",578.00,5.54,0.00,5.54,5.54,5.54,5.54,5.54
"(24, 27]",375.00,5.46,0.00,5.46,5.46,5.46,5.46,5.46
"(3, 4]",372.00,4.45,0.45,3.92,4.23,4.23,4.62,5.15
"(4, 5]",830.00,4.53,0.66,3.69,4.23,4.38,4.92,5.69
"(5, 6]",822.00,4.66,1.13,3.62,3.62,5.08,5.08,6.38
"(6, 8]",2403.00,4.36,0.47,3.92,4.08,4.08,4.62,5.54
"(8, 10]",1762.00,5.13,0.71,4.23,4.23,5.23,6.00,6.00


In [31]:
de.groupby(['Age_group'])['SelfReg_NegAffIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(10, 12]",1600.00,4.33,0.85,3.00,3.50,4.75,4.75,5.67
"(12, 15]",578.00,5.50,0.00,5.50,5.50,5.50,5.50,5.50
"(24, 27]",375.00,4.42,0.00,4.42,4.42,4.42,4.42,4.42
"(3, 4]",372.00,4.29,0.79,3.17,3.17,4.50,4.83,5.08
"(4, 5]",830.00,4.36,0.58,3.17,4.33,4.33,4.75,5.25
"(5, 6]",822.00,3.78,1.41,2.83,2.83,3.33,3.33,6.33
"(6, 8]",2403.00,4.66,0.46,3.33,4.58,4.75,4.83,5.42
"(8, 10]",1762.00,3.86,1.54,2.42,2.42,2.75,5.83,5.83


In [32]:
de.groupby(['Age_group'])['SelfReg_EffContIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(10, 12]",1600.00,5.36,1.14,3.83,4.75,4.75,6.75,7.08
"(12, 15]",578.00,5.67,0.00,5.67,5.67,5.67,5.67,5.67
"(24, 27]",375.00,5.83,0.00,5.83,5.83,5.83,5.83,5.83
"(3, 4]",372.00,6.15,0.29,5.83,5.83,6.42,6.42,6.42
"(4, 5]",830.00,5.12,0.69,4.25,4.67,5.08,5.58,6.50
"(5, 6]",822.00,5.29,0.27,4.92,4.92,5.33,5.33,5.67
"(6, 8]",2403.00,5.10,0.66,4.25,4.58,5.25,5.33,6.08
"(8, 10]",1762.00,5.27,0.89,4.08,4.08,5.17,5.58,6.58


In [33]:
de.PIN.nunique()

37